# UNISYS ハイスキルエンジニア プログラミング課題
日本ユニシスの選考で使用するNotebook．<br>
# 今回の方針
- titanicで挫折して以来のKaggleであるので，いろんなNotebookを参考にしてできるだけ予測精度を上げる．
- テーブルデータや需要の予測は今までやったことがないので初心者でも簡単に予測できるfbprophetで実装する

# 準備
## 今回使用するPythonパッケージのインポート

In [ ]:
import numpy as np 
import pandas as pd 
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from fbprophet import Prophet

from joblib import Parallel, delayed

## 使用するデータのロード
### memoryの節約
計算のコストやmemoryの使用量を削減するために型のキャストを行う．<br>
memoryの削減は[このNotebook](https://www.kaggle.com/omershect/learning-pytorch-lstm-deep-learning-with-m5-data)を参考にした．<br>
この関数は時間がかかるため，tqdmを用いてプログレスバーの表示機能を追加した．

In [ ]:
def reduce_mem_usage(new_data: pd.DataFrame, verbose: bool=True) -> pd.DataFrame:
    """
    メモリの使用量を削減する

    Parameters
    ----------
    new_data : pd.DataFrame
        memoryを削減したいテーブルデータ
    verbose : bool
        memory削減量の標準出力の有無

    Returns
    -------
    new_data : new_data.DataFrame
        memory削減後テーブルデータ
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']    # 想定されるデータの型
    start_mem = new_data.memory_usage().sum() / 1024**2    # 初期のメモリの使用量
    
    # 列ごとの型指定を行うためのループ
    for col in tqdm(new_data.columns):
        col_type = new_data[col].dtypes
        # 型が数値型の場合はキャストする
        if col_type in numerics: 
            c_min = new_data[col].min()
            c_max = new_data[col].max()
            # 型がint型の場合
            if str(col_type)[:3] == 'int':
                # 最小値と最大値の値に応じて適宜キャスト
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    new_data[col] = new_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    new_data[col] = new_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    new_data[col] = new_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    new_data[col] = new_data[col].astype(np.int64)  
            # 型がfloat型の場合
            else:
                # 最小値と最大値の値に応じて適宜キャスト
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    new_data[col] = new_data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    new_data[col] = new_data[col].astype(np.float32)
                else:
                    new_data[col] = new_data[col].astype(np.float64)    
    end_mem = new_data.memory_usage().sum() / 1024**2    # 削減後のメモリの使用量
    if verbose:    # 標準出力
        print(f'Mem. usage decreased to {end_mem:5.2f} Mb ({(100 * (start_mem - end_mem) / start_mem):.1f}% reduction)')
    return new_data

### データの事前知識
- **calendar.csv**<br>
Contains information about the dates on which the products are sold.<br>
(製品が販売された日付の情報を含む.)<br>
- **sales_train_validation.csv**<br>
Contains the historical daily unit sales data per product and store [d_1 - d_1913]<br>
(製品別・店舗別の過去の日次販売台数データ[d_1 - d_1913]を収録.)<br>
- **sample_submission.csv**<br>
The correct format for submissions. Reference the Evaluation tab for more info.<br>
(提出するための正しいフォーマットです．詳細は「評価」タブを参照してください．)<br>
- **sell_prices.csv**<br>
Contains information about the price of the products sold per store and date.<br>
(店舗ごと・日付ごとの販売商品の価格情報を収録．)<br>
- **sales_train_evaluation.csv**<br>
Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)<br>
(セールスを含む [d_1 - d_1941] （パブリック・リーダーボードに使用されるラベル）)<br>

先ほど作った関数を利用してデータを読み込む

In [ ]:
# load data with reducing memory usage
calender = reduce_mem_usage(pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv'))
validate = reduce_mem_usage(pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv'))
sample_submission = reduce_mem_usage(pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv'))
prices = reduce_mem_usage(pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv'))
evaluate = reduce_mem_usage(pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv'))

# EDA
## データの確認
どのようなテーブルデータが入っているかや欠損値がないかなどを確認しないことには可視化もできないのでまずはデータを表示してみる．
### calender.csvについて

In [ ]:
# show head of dataframe
calender.head()

#### headerの内訳
- date           : ハイフンで区切ってある日付 (xxxx-yy-zz)<br>
- wm_yr_wl       : 土曜日に+1され，1年毎に100の位が+1され10の位以下はリセットされる<br>
- weekday        : 曜日 (Saturday, Sunday, ...)<br>
- wday           : 曜日を数字で表したもの (Sat=1, Sun=2, ...)<br>
- month, year    : それぞれ月，年 (1, 2, ... | 2011, 2012, ...)<br>
- d              : 通し番号 (d1, d2, ...)<br>
- event_name_x   : イベントの名前，1日に2つある場合はevent_name_2に入力してある (SuperBowl, ValentinesDay)<br>
- event_type_x   : イベントのタイプ (Sporting, Cultural, National)<br>
- snap_XX        : 金券のようなもの．CA(California)，TX(Texas)，WI(Wisconsin)の3種類

In [ ]:
# null_check
calender.isnull().sum().sort_values(ascending = False)

`event_name`と`event_type`には`NaN`が含まれているので扱い注意
### sales_train_validation.csvについて

In [ ]:
# show head of dataframe
validate.head()

#### headerの内訳
- id             : カテゴリー・店舗などの情報を持ったid (xxxx-yy-zz)<br>
- item_id        : アイテムのid<br>
- dept_id        : 部門のid<br>
- cat_id         : カテゴリーのid<br>
- store_id       : 店舗id<br>
- state_id       : 州のid<br>
- d_x            : x日目に売れた数

In [ ]:
# null check
validate.isnull().sum().sort_values(ascending = False)

### sample_submission.csvについて

In [ ]:
# show head of dataframe
sample_submission.head()

#### headerの内訳
- id             : カテゴリー・店舗などの情報を持ったid (xxxx-yy-zz)<br>
- F1~28          : その日の売り上げ<br>

### sell_prices.csvについて

In [ ]:
# show head of dataframe
prices.head()

#### headerの内訳
- store_id       : 店舗id<br>
- item_id        : アイテムのid<br>
- wm_yr_wk       : 土曜日に+1され，1年毎に100の位が+1され10の位以下はリセットされる
- sell_price     : 商品の値段

In [ ]:
# null check
prices.isnull().sum().sort_values(ascending = False)

### sales_train_evaluation.csvについて

In [ ]:
# show head of dataframe
evaluate.head()

In [ ]:
# null check
evaluate.isnull().sum().sort_values(ascending = False)

## データの可視化
どのような形式でデータが格納されているかは確認できた．<br>
普段Notebookは使わないが，matplotlibよりplotlyの方がグラフの拡大や値の参照などができ便利そう．<br>
勉強がてら，今回はplotlyでグラフの表示をしてみる
### **sales_train_validation.csv**について
#### 適当なデータでの需要

In [ ]:
# 日付データは今後の可視化で必要なので配列として抜き出す
date_list = calender.date.to_list()

# validateのdfのcolumnsからd_のものを抜き出す
ids = sorted(list(set(validate['id'])))
d_cols = [c for c in validate.columns if 'd_' in c]

# 適当にデータを抜き出す
x_1 = validate.loc[10, d_cols].to_list()
x_2 = validate.loc[100, d_cols].to_list()
x_3 = validate.loc[1000, d_cols].to_list()

# plotlyを利用した可視化
fig = go.Figure()
fig.add_trace(go.Scatter(x=date_list[:len(x_1)], y=x_1,
                    mode='lines', name=validate.iloc[10,0],opacity=0.5))

fig.add_trace(go.Scatter(x=date_list[:len(x_2)], y=x_2,
                    mode='lines', name=validate.iloc[100,0],opacity=0.5))

fig.add_trace(go.Scatter(x=date_list[:len(x_3)], y=x_3,
                    mode='lines', name=validate.iloc[1000,0],opacity=0.5))

# データスタンプの存在するグラフを扱いやすくする
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="month", step="month", stepmode="backward"),
                dict(count=6, label="6month", step="month", stepmode="backward"),
                dict(count=1, label="year", step="year", stepmode="backward"),
                dict(step="all")])),
        rangeslider=dict(visible=True),
        type="date"))
fig.show()

初めてのplotlyだが，かなり便利な予感がする．<br>
とりあえず，適当に取り出したサンプルから分かることは以下のこと
- HOBBIE_1_011のように需要に波がある商品があるということ
- HOBBIE_1_105のようにある程度通年需要がある商品があること
- いずれにしてもかなりノイジーなデータであること

次は店舗ごとの需要の合計を見てみる<br>
これを見ればもう少し傾向が見えてくるだろう<br>
この後で数回グラフを表示するので，関数を作っておく

In [ ]:
def line_graph(df: pd.DataFrame, opacity=1, use_rolling: bool=True, window: int=7, colors=px.colors.qualitative.Plotly) -> None:
    """
    折れ線グラフを表示する

    Parameters
    ----------
    df : pd.DataFrame
        グラフを表示させたいテーブルデータ
    opacity : int
        グラフの濃さ
    use_rolling : bool
        移動平均のグラフを表示するかどうか
    window : int
        移動平均を利用する際の窓幅
    colors :
        グラフの色
    """
    org_opacity = opacity
    fig = go.Figure()
    data_length = len(df.columns)
    for i, (idx, row) in enumerate(df.iterrows()):
        # 移動平均
        if use_rolling:
            fig.add_trace(go.Scatter(x=date_list[:data_length], y=row.rolling(window).mean(), line=dict(color=colors[i]),
                            mode='lines', name=f'{idx}_rolling',opacity=opacity))
            opacity = 0.2
        fig.add_trace(go.Scatter(x=date_list[:data_length], y=row, line=dict(color=colors[i]),
                    mode='lines', name=idx,opacity=opacity))
        opacity=org_opacity
    # データスタンプの存在するグラフを扱いやすくする
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="month", step="month", stepmode="backward"),
                    dict(count=6, label="6month", step="month", stepmode="backward"),
                    dict(count=1, label="year", step="year", stepmode="backward"),
                    dict(step="all")])),
            rangeslider=dict(visible=True),
            type="date"))
    fig.show()

#### 店舗ごとの需要

In [ ]:
# 店舗毎の合計を求める
store_sales = validate.groupby('store_id').sum()
# グラフの表示
line_graph(store_sales, use_rolling=True)

店舗ごとに需要の合計と1週間の移動平均をプロットしてみたところ以下のような特徴が見えてきた<br>
- 12/25クリスマスはwalmartがクリスマス休日であることから毎年売れ行きが0近くになっている(なぜか売れている商品もあるが．．．)
- 1週間ごとの周期があり，週末の需要が高い
- 1か月ごとの周期があり，月初めの需要が高い
- 1年ごとの周期があり，夏の需要が高い
- 全体的に売り上げが伸びている．
- 売り上げが増加する時期もあれば減少する時期もある(需要のトレンド)

全ての店舗がこれに当てはまるわけではないが，直感とあった傾向であるので予測モデルに組み込みたい<br>
同様にカテゴリごとのグラフもプロットしてみる
#### カテゴリごとの需要

In [ ]:
# カテゴリ毎の合計を求める
dept_sales = validate.groupby('dept_id').sum()
# グラフの表示
line_graph(dept_sales, use_rolling=True)

周期に関しては店舗ごとのグラフと同様の傾向．<br>
カテゴリ別で特有なのは
- HOUSEHOLD_1が年々売り上げを伸ばしていること
- FOOD_3の売り上げが顕著に高い
- HOBBIES_2はほぼ売れない

周期性は見えてきたので今度は棒グラフで売り上げなどの傾向を見てみる<br>
こちらはボックスプロットで表示したいので，こちらも関数を作っておく

In [ ]:
def box_plot(df: pd.DataFrame, colors=px.colors.qualitative.Plotly) -> None:
    """
    折れ線グラフを表示する

    Parameters
    ----------
    df : pd.DataFrame
        グラフを表示させたいテーブルデータ
    colors :
        グラフの色
    """
    fig = go.Figure()
    for i, (idx, row) in enumerate(df.iterrows()):
        fig.add_trace(go.Box(
            y=row,
            name=idx,
            marker_color=colors[i],
            boxmean=True # represent mean
        ))
    fig.show()

#### 店舗ごとの需要

In [ ]:
box_plot(store_sales)

店舗による需要の差は大きい．<br>
例えばCA_3は平均して6000個の需要が毎日あるのに対し，CA_4は2000個の需要である
#### カテゴリごとの需要

In [ ]:
box_plot(dept_sales)

カテゴリによる需要の差は大きい．<br>
例えばFOODS_3は平均して17000個の需要が毎日あるのに対し，HOBBIES_2は0個の需要である．<br>
また，データのばらつきもFOOD_3は大きいがそのほかは比較的小さい<br>

以上のように店舗やカテゴリ特有の需要があることが分かるため，機械学習で扱う際は店舗・カテゴリごとに標準化したい．
# 機械学習
ある程度EDAが終わり，3つの周期性があることや土日の売り上げが高いことなどが分かった．<br>
このような情報をもとに予測を行いたい．<br>
まずはテーブルデータを機械学習で扱いやすい形に変形していく．<br>
## データの前処理
機械学習では入力の値を標準化することが多い．<br>
正規化するものや[-1,1]の範囲に李スケールするものなどがある．<br>
今回はボックスプロットからわかった店舗・カテゴリごとのばらつきを考慮して，店舗・カテゴリごとの需要割合で標準化する([参考](https://www.kaggle.com/raghvenbhati/prophet-forecasts))．<br>
まずは`evaluate`のデータから店舗とカテゴリでグルーピングした集合テーブルを作る（分母となる）

In [ ]:
sum_group = evaluate.groupby(['dept_id','store_id'], as_index=False).sum()
_denominator_ave = sum_group[['dept_id','store_id']]

denominator_ave = pd.DataFrame(sum_group.iloc[:,1550:].mean(axis=1),columns=['denominator_ave'])
denominator_ave = pd.concat([_denominator_ave, denominator_ave], axis=1)
denominator_ave.head()

次は分子を計算する

In [ ]:
index = evaluate[['id','dept_id','store_id']]
numerator_ave = pd.DataFrame(evaluate.iloc[:,1550:].mean(axis=1),columns=['numerator_ave'])
numerator_ave = pd.concat([index, numerator_ave], axis=1)
numerator_ave.head()

割合を追加する

In [ ]:
fraction = pd.merge(numerator_ave, denominator_ave, on =['dept_id','store_id'])
fraction['fraction_ave'] = fraction['numerator_ave']/fraction['denominator_ave']
fraction

fbprophetでは不定期に表れる休日を考慮した予測ができるので，イベントが起こった日付を取り出す

In [ ]:
holiday1 = calender.iloc[858:1969,].loc[calender['event_name_1'].notnull()][['event_name_1','date']].rename(columns={'event_name_1':'holiday','date':'ds'})
holiday2 = calender.iloc[858:1969,].loc[calender['event_name_1'].notnull()][['event_type_1','date']].rename(columns={'event_type_1':'holiday','date':'ds'})
holiday3 = calender.iloc[858:1969,].loc[calender['event_name_2'].notnull()][['event_name_2','date']].rename(columns={'event_name_2':'holiday','date':'ds'})
holiday4 = calender.iloc[858:1969,].loc[calender['event_name_2'].notnull()][['event_type_2','date']].rename(columns={'event_type_2':'holiday','date':'ds'})
holidays = pd.concat((holiday1, holiday2,holiday3,holiday4))
holidays.head()

## 学習と予測
データの前処理が終わったので，実際に学習と予測に移る．<br>
今回はfbprophetを利用して予測を行う．<br>
今回データ分析から見えた情報を含む以下の条件を追加でモデルに組み込む
- EDAで発見できたデータの周期
- 休日やイベントがある日
- 年月日の情報（年々売り上げが伸びていたり，トレンドがあるため）

In [ ]:
def prophet(i):
    """
    需要の予測をする関数

    Parameters
    ----------
    i : int
        予測したいpandasの行

    Returns
    -------
    pred: pd.DataFrame
        予測テーブルデータ
    """
    # 予測モデルのインスタンス化
    # yealy_seasonality大きいほど周期性を強く考慮する
    # 休日のデータはここで引数として入力する
    m = Prophet(yearly_seasonality=24, holidays=holidays)
    
    # 周期性の指定(EDAで分かった3つの周期を考慮する)
    m.add_seasonality(name='yearly', period=365, fourier_order=10)
    m.add_seasonality(name='monthly', period=365/12, fourier_order=10)
    m.add_seasonality(name='weekly', period=7, fourier_order=5)
    
    # 予測したいモデルの入力
    # ds:datestamp y:正解ラベル
    tsdf = pd.DataFrame({
      'ds': pd.to_datetime(calender.iloc[858:1941,]['date'].reset_index(drop=True)),
      'y': sum_group.iloc[i,860:1943].reset_index(drop=True),
    })
    # 説明変数の追加（年月日の情報を追加する）
    tsdf['wday']=calender.iloc[858:1941,]['wday'].reset_index(drop=True)
    tsdf['month']=calender.iloc[858:1941,]['month'].reset_index(drop=True)
    tsdf['year']=calender.iloc[858:1941,]['year'].reset_index(drop=True)
    #m.add_regressor('sell_price')
    m.add_regressor('wday')
    m.add_regressor('month')
    m.add_regressor('year')
    # 学習
    m.fit(tsdf)
    # 予測する範囲を指定
    future = m.make_future_dataframe(periods=28)
    future['wday']=calender.iloc[858:1969,]['wday'].reset_index(drop=True)
    future['month']=calender.iloc[858:1969,]['month'].reset_index(drop=True)
    future['year']=calender.iloc[858:1969,]['year'].reset_index(drop=True)
    # 予測
    forecast = m.predict(future)
    pred = pd.DataFrame(forecast.iloc[1083:1112,]['yhat'])
    pred['dept_id']=denominator_ave.iloc[i,]['dept_id']
    pred['store_id']=denominator_ave.iloc[i,]['store_id']
    return pred

In [ ]:
# fbprophetはGPUに対応していないのでCPUの並列計算で処理速度を担保する
pred = Parallel(n_jobs=-1)(delayed(prophet)(i) for i in range(denominator_ave.shape[0]))

# 各商品ごとに推測された需要を組み合わせる
preds = pd.concat(pred[0:70])
preds['period']=preds.index
preds['period']=preds['period']-1082

# 提出のフォーマットに適した形に変形する
pivot_preds=preds.pivot_table(index=['dept_id','store_id'], columns='period', values='yhat')
prev_submission = pd.merge(fraction, pivot_preds, on =['dept_id','store_id'])
for i in range(28):
    # 標準化した値をもとに戻す
    prev_submission.iloc[:,(6+i)] = prev_submission.iloc[:,(6+i)]*prev_submission['fraction_ave']

submission=prev_submission[['id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]]
submission.columns = sample_submission.columns
submission_index=sample_submission[['id']]
submission = pd.merge(submission_index, submission, on = 'id', how = 'left')
submission = submission.fillna(0)    # 欠損値を0に置き換える
submission.to_csv('submission.csv',index=False)    # 提出用のcsvファイルを作成する